### Train the TSVM on the Dataset 

### 1. I have implemented the TSVM using the paper 

In [2]:
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from mlp_multi import MLP  
from svm import MulticlassTSVM  

# Load the dataset CIFAR-10
data = load_dataset("uoft-cs/cifar10")

# Flatened the images into Arrray for traing and testing
def preprocess_img(imgs):
    flattened_imgs = []
    for img in imgs:
        flattened_imgs.append(np.array(img).flatten())
    flattened_imgs_array = np.array(flattened_imgs)
    return flattened_imgs_array

# Intializing the trainging and testing data to corresponding Variable
X_train = preprocess_img(data['train']['img'])
y_train = np.array(data['train']['label'])
X_test = preprocess_img(data['test']['img'])
y_test = np.array(data['test']['label'])

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Split the data into labled and unlabled set for training and Testing
X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

# One-hot encode the labels for MLP training
n_classes = len(np.unique(y_train))
y_labeled_one_hot = np.eye(n_classes)[y_labeled]

# Intialising the parameters of TSVM for multi-classification
multi_tsvm = MulticlassTSVM(
    n_classes=n_classes,
    max_iter=50,
    learning_rate=0.01,
    lambda_param=0.01,
    lambda_u=0.1,
    itr=100
)

print("Training TSVM...")
multi_tsvm.fit(X_labeled, y_labeled, X_unlabeled)

print("Testing TSVM...")
y_pred = multi_tsvm.predict(X_test)

# Evaluate accuracy of the Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

precision = precision_score(y_test, y_pred, average='macro')  # Options: 'macro', 'micro', 'weighted', 'binary'
print(f"Precision: {precision:.2f}")

# Recall
recall = recall_score(y_test, y_test, average='macro')
print(f"Recall: {recall:.2f}")

# F1-Score
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1-Score: {f1:.2f}")


Training TSVM...
Testing TSVM...
Accuracy: 0.1000
Precision: 0.01
Recall: 1.00
F1-Score: 0.02


/Users/akashrana/Downloads/S3VM/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 2. This is the implentation of My hybrid Model - MLP + TSVM

In [4]:
# Initialize the MLP model
mlp = MLP(input_size=X_labeled.shape[1], n_classes=n_classes)

# Load pre-trained weights
weights_file = "multi_mlp_weights.pkl"
mlp.load_weights(weights_file)

# Fine Tunning of the MLP Model for efficient featre extraction
n_epochs = 100 
learning_rate = 0.01

print("Fine Tuning the MLP...")
for epoch in range(n_epochs):
    outputs = mlp.feed_forward_block(X_labeled)
    loss = -np.mean(np.sum(y_labeled_one_hot * np.log(outputs + 1e-7), axis=1))
    mlp.backpropagation(y_labeled_one_hot, learning_rate=learning_rate)

# Extracting the features from data using pre-trained MLP
X_labeled_features = mlp.extract_features(X_labeled)
X_unlabeled_features = mlp.extract_features(X_unlabeled)
X_test_features = mlp.extract_features(X_test)

# Intializing the Parameters of TSVM
multi_tsvm = MulticlassTSVM(
    n_classes=n_classes,
    max_iter=50,
    learning_rate=0.01,
    lambda_param=0.01,
    lambda_u=0.1,
    itr=100
)

print("Training TSVM...")
multi_tsvm.fit(X_labeled_features, y_labeled, X_unlabeled_features)

print("Testing TSVM...")
y_pred = multi_tsvm.predict(X_test_features)

# Evaluate accuracy on the test data of the Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

precision = precision_score(y_test, y_pred, average='macro')  # Options: 'macro', 'micro', 'weighted', 'binary'
print(f"Precision: {precision:.2f}")

# Recall
recall = recall_score(y_test, y_pred, average='macro')
print(f"Recall: {recall:.2f}")

# F1-Score
f1 = f1_score(y_test, y_pred, average='macro')
print(f"F1-Score: {f1:.2f}")

Weights loaded from multi_mlp_weights.pkl
Fine Tuning the MLP...
Training TSVM...
Testing TSVM...
Accuracy: 0.1000
Precision: 0.01
Recall: 0.10
F1-Score: 0.02


/Users/akashrana/Downloads/S3VM/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
